In [0]:
USE SCHEMA project_silver

## DimAirlines

In [0]:
CREATE TABLE IF NOT EXISTS DimAirlines (
  airline_key BIGINT GENERATED ALWAYS AS IDENTITY,
  iata_code STRING,
  airline STRING,
  valid_from DATE,
  valid_to DATE,
  is_current BOOLEAN
)
USING DELTA;

In [0]:
SELECT * FROM project_bronze.airlines

iata_code,airline
UA,United Air Lines Inc.
AA,American Airlines Inc.
US,US Airways Inc.
F9,Frontier Airlines Inc.
B6,JetBlue Airways
OO,Skywest Airlines Inc.
AS,Alaska Airlines Inc.
NK,Spirit Air Lines
WN,Southwest Airlines Co.
DL,Delta Air Lines Inc.


In [0]:
MERGE INTO DimAirlines AS d
USING (
    SELECT
        iata_code,
        airline
    FROM project_bronze.airlines
) AS a
ON d.iata_code = a.iata_code AND d.is_current = true
 
WHEN MATCHED AND (
  d.airline <> a.airline
) THEN
  UPDATE SET
    d.valid_to   = current_timestamp(),
    d.is_current = false
 
WHEN NOT MATCHED THEN
  INSERT (
    iata_code, airline, valid_from, valid_to, is_current
  )
  VALUES (
    a.iata_code, a.airline, current_timestamp(), DATE '9999-12-31', true
  );


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
14,0,0,14


In [0]:
SELECT * FROM project_silver.dimairlines

airline_key,iata_code,airline,valid_from,valid_to,is_current
1,UA,United Air Lines Inc.,2025-12-04,9999-12-31,true
2,AA,American Airlines Inc.,2025-12-04,9999-12-31,true
3,US,US Airways Inc.,2025-12-04,9999-12-31,true
4,F9,Frontier Airlines Inc.,2025-12-04,9999-12-31,true
5,B6,JetBlue Airways,2025-12-04,9999-12-31,true
6,OO,Skywest Airlines Inc.,2025-12-04,9999-12-31,true
7,AS,Alaska Airlines Inc.,2025-12-04,9999-12-31,true
8,NK,Spirit Air Lines,2025-12-04,9999-12-31,true
9,WN,Southwest Airlines Co.,2025-12-04,9999-12-31,true
10,DL,Delta Air Lines Inc.,2025-12-04,9999-12-31,true


## DimAirport

In [0]:
CREATE TABLE IF NOT EXISTS DimAirports (
  airport_key BIGINT GENERATED ALWAYS AS IDENTITY,
  iata_code STRING,
  airport STRING,
  city STRING,
  state STRING,
  country STRING,
  latitude FLOAT,
  longitude FLOAT,
  valid_from DATE,
  valid_to DATE,
  is_current BOOLEAN
)
USING DELTA;

In [0]:
MERGE INTO DimAirports AS d
USING (
    SELECT
        iata_code,
        airport,
        city,
        state,
        country,
        latitude,
        longitude
    FROM project_bronze.airports
) AS a
ON d.iata_code = a.iata_code AND d.is_current = true
 
WHEN MATCHED AND (
  d.airport <> a.airport
  OR d.city <> a.city
  OR d.state <> a.state
  OR d.country <> a.country
  OR d.latitude <> a.latitude
  OR d.longitude <> a.longitude

) THEN
  UPDATE SET
    d.valid_to   = current_date(),
    d.is_current = false
 
WHEN NOT MATCHED THEN
  INSERT (
    iata_code, airport, city, state, country, latitude, longitude, valid_from, valid_to, is_current
  )
  VALUES (
    a.iata_code, a.airport, a.city, a.state, a.country, a.latitude, a.longitude, current_date(), DATE '9999-12-31', true
  );


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
322,0,0,322


## DimDate

In [0]:
%python
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df_dim_date = (
    spark.read.table("project_bronze.flights")
    .select("year", "month", "day", "day_of_week")
    .withColumn(
        "full_date",
        F.concat(
            F.lpad(F.col("year"), 4, "0"),
            F.lit("-"),
            F.lpad(F.col("month"), 2, "0"),
            F.lit("-"),
            F.lpad(F.col("day"), 2, "0")
        )
    )
    .distinct()
)

w = Window.orderBy("full_date")

df_dim_date = df_dim_date.withColumn("date_key", F.row_number().over(w))
df_dim_date = df_dim_date.select("date_key", "full_date", "year", "month", "day", "day_of_week")

df_dim_date.write.mode("overwrite").saveAsTable("DimDate")

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(
